### First - You must install all the required libraries mentioned in "requirements.txt" file - One time requirement

##### - To install libraries -> Run "requirements.txt" file
##### - Command to run in Terminal -> pip install -r requirements.txt

#### - If not able to run in terminal, go to "requirements.txt" file in your directory and copy each library and run individually in terminal and install using command - pip install "library"
- E.g. pip install selenium=4.15.2
##### Mark Woodamsnee edit
##### Or you can do it in this file only with command E.g. !pip install selenium=4.15.2

### Libraries to import

In [1]:
import selenium
import sys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import bs4
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
import time
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Caution - This file you should Run only once in a month

## Full Code to retrieve data from 8 websites into .csv files

#### - To run the cell - select the particular cell and press Ctrl+Enter or you can choose from left arrow button on the cell

In [2]:
options = webdriver.ChromeOptions()
#options.add_argument("--start-maximized")
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.page_load_strategy = "none"

# Get today's date
today = datetime.today()
# Get the month name from today's date
month_name = today.strftime('%B')

print("Today's month:", month_name)

keywords_ignore=['Teaching', 'Instructor','Research Scientist','Lecturer','Research Assistant','Research Engineer','Teaching Professor',
                 'Teaching Assistant Professor','Emeritus','Emerita','Professor Emeritus','Professor Emerita','Adjunct','Adjunct Professor',
                 'Professor of the Practice','Coordinator','Instrument Maker','System Support Engineer','Academic Advisor',
                 'Mechanical Engr','Communications Officer','Research Technician','Research Investigator','Visiting Assistant Professor',
                 'Department Affiliate','Director of Human Resources','Manager','Courier','Helluva Engineer','HR Consultant','Academic Assistant',
                 'Support Professional','Academic Professional Chair in Communication Skills','Financial Administrator','Senior Engineer',
                 'Director IT','Associate Chair for Inclusive Excellence','Administrative Professional','Director of Student Engagement and Success',
                 'Director of Business Operations','Mail Clerk','Director of Design & Innovation','Financial Admin','Administrator Lead',
                 'Director of Development','Electrical Engineer','Administrative Supervisor','Support Prof','Director of Laboratory Development',
                 'Machine Shop Supervisor','Senior Academic Professional','Academic Professional','Facilities Assistant','Admin Professional',
                 'Mechanical Engineer','Communications Officer','Financial Mgr','Grants Administrator','Development Assistant',
                 'Application Developer','Administrative Officer','Visiting Professor','Mechanical Technician','Visiting Scholar']



Today's month: September


In [7]:
# All functions for 8 websites

def Caltech(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#    options = Options()
#    options.add_experimental_option("detach", True)
#    driver = webdriver.Chrome(options=options)

    driver.get("https://mce.caltech.edu/people")
    time.sleep(5)

    # Parse the HTML content of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "name" (containing professor names)
    names = soup.find_all('div', class_='person-teaser__title')

    # Find all div elements with class "title" (containing professor titles/profiles)
    titles = soup.find_all('div', class_='person-teaser__job-title')

    professor_names = []
    professor_profiles = []

    # Iterate over the professors and print their names and profiles
    for name, title in zip(names, titles):
        professor_names.append(name.text.strip())
        professor_profiles.append(title.text.strip())
        
    # Create a dataframe to store the data
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    os.makedirs(month_name)

    professors_df.to_csv('./'+month_name+'/'+'caltechprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'caltechprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]
    
    df.to_csv('./'+month_name+'/'+'caltechprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))
    
    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'
    

    df=df[['Professor Name','Professor Profile','Faculty']]
    os.mkdir('./'+month_name+'/'+'files')

    df.to_csv('./'+month_name+'/'+'files/caltechprofessors_prof_category.csv',index=False)
    print('Done for Caltech')


def Georgia(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://www.me.gatech.edu/faculty")
    time.sleep(5)

    professor_names = []
    professor_profiles = []

    def data(driver):
        # Parse the HTML content of the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all div elements with class "name" (containing professor names)
        names = soup.find_all('div', class_='faculty-name')

        # Find all div elements with class "title" (containing professor titles/profiles)
        titles = soup.find_all('div', class_='faculty-title')

        # Iterate over the professors and print their names and profiles
        for name, title in zip(names, titles):
            professor_names.append(name.text.strip())
            professor_profiles.append(title.text.strip())
        
        return professor_names,professor_profiles

    count=1
    for i in range(50):
        professor_names,professor_profiles = data(driver)
        print(len(professor_names))

        try:
            print(count)

            next_button =driver.find_element(By.CSS_SELECTOR, "a[title='Go to next page']")
            # Click on the "Next" button
            ActionChains(driver).move_to_element(next_button).click().perform()

            # Wait for the page to load
            time.sleep(10)

            # Increment the count
            count += 1
        except:
            print("Next button not found or reached end of pages")
            break


    # Create a dataframe to store the data
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    professors_df.to_csv('./'+month_name+'/'+'georgiaprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'georgiaprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]

    df.to_csv('./'+month_name+'/'+'georgiaprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))
    
    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'

    df=df[['Professor Name','Professor Profile','Faculty']]
    
    df.to_csv('./'+month_name+'/'+'files/georgiaprofessors_prof_category.csv',index=False)
    print('Done for Georgia')


def Michigan(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://me.engin.umich.edu/people/faculty/")
    time.sleep(5)

    # Parse the HTML content of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "name" (containing professor names)
    names = soup.find_all('span', class_='faculty-name')

    # Find all div elements with class "title" (containing professor titles/profiles)
    titles = soup.find_all('span', class_='faculty-titles')

    professor_names = []
    professor_profiles = []

    # Iterate over the professors and print their names and profiles
    for name, title in zip(names, titles):
        professor_names.append(name.text.strip())
        professor_profiles.append(title.text.strip())
        
    # Create a dataframe to store the data
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    professors_df.to_csv('./'+month_name+'/'+'michiganprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'michiganprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]

    df.to_csv('./'+month_name+'/'+'michiganprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))

    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'

    df=df[['Professor Name','Professor Profile','Faculty']]
    
    df.to_csv('./'+month_name+'/'+'files/michiganprofessors_prof_category.csv',index=False)
    print('Done for Michigan')

def MIT(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://meche.mit.edu/people")
    time.sleep(5)

    # Parse the HTML content of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "name" (containing professor names)
    names = soup.find_all('span', class_='name')

    # Find all div elements with class "title" (containing professor titles/profiles)
    titles = soup.find_all('span', class_='title')

    professor_names = []
    professor_profiles = []

    # Iterate over the professors and print their names and profiles
    for name, title in zip(names, titles):
        professor_names.append(name.text.strip())
        professor_profiles.append(title.text.strip())
        
    # Create a dataframe to store the data
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    professors_df.to_csv('./'+month_name+'/'+'mitprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'mitprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]

    df.to_csv('./'+month_name+'/'+'mitprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))

    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'

    df=df[['Professor Name','Professor Profile','Faculty']]

    df.to_csv('./'+month_name+'/'+'files/mitprofessors_prof_category.csv',index=False)
    print('Done for MIT')

def Purdue(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://engineering.purdue.edu/ME/People/faculty.html")
    time.sleep(5)

    # Parse the HTML content of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "name" (containing professor names)
    names = soup.find_all('div', class_='col col-xs-12 col-md-7 list-name')

    # Find all div elements with class "title" (containing professor titles/profiles)
    titles = soup.find_all('div', class_='people-list-title')

    professor_names = []
    professor_profiles = []

    # Iterate over the professors and print their names and profiles
    for name, title in zip(names, titles):
        professor_names.append(name.text.strip())
        professor_profiles.append(title.text.strip())
        
    # Create a dataframe to store the data
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    a = lambda prof: prof.split('\n')[0]
    professors_df['Professor Name'] = professors_df['Professor Name'].apply(a)

    professors_df.to_csv('./'+month_name+'/'+'purdueprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'purdueprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]

    df.to_csv('./'+month_name+'/'+'purdueprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))

    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'

    df=df[['Professor Name','Professor Profile','Faculty']]
    
    df.to_csv('./'+month_name+'/'+'files/purdueprofessors_prof_category.csv',index=False)
    print('Done for Purdue')

def Standford(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://me.stanford.edu/people/faculty")
    time.sleep(5)

    # Parse the HTML content of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "name" (containing professor names)
    names = soup.find_all('span', class_='field-content')

    # Find all div elements with class "title" (containing professor titles/profiles)
    titles = soup.find_all('div', class_='field-content')

    professor_names = []
    professor_profiles = []

    # Iterate over the professors and print their names and profiles
    for name, title in zip(names, titles):
        professor_names.append(name.text.strip())
        professor_profiles.append(title.text.strip())
        
    # Create a dataframe to store the data
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    professors_df.to_csv('./'+month_name+'/'+'stanfordprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'stanfordprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]
        
    df.to_csv('./'+month_name+'/'+'stanfordprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))

    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'

    df=df[['Professor Name','Professor Profile','Faculty']]

    df.to_csv('./'+month_name+'/'+'files/stanfordprofessors_prof_category.csv',index=False)
    print('Done for Stanford')

def UC_Berkeley(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://me.berkeley.edu/faculty/")
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "name" (containing professor names)
    a_tags = soup.find_all('a', class_='_self cvplbd')
    href_list = []

    # Extract the href attribute from each <a> tag and store in the list
    for a_tag in a_tags:
        href_list.append(a_tag['href'])


    professor_names=[]
    professor_profiles=[]
    print(len(href_list))

    for i,link in enumerate(href_list):
        print(i, href_list[i])
        options = webdriver.ChromeOptions()
        #options.add_argument("--start-maximized")
        options.add_argument("--headless")
        #options.page_load_strategy = "none"
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        driver.get(link)
        
        # Parse the HTML content of the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        names = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h1[@class="entry-title"]')))


        titles = driver.find_element("tag name", "h3")

        # Find the p tag within the h3 tag
        if i == 12:
            print("skipped i = 12")
        else:
            p_element = titles.find_element("css selector", "p")

            professor_names.append(names.text)
            professor_profiles.append(p_element.text)
        
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    professors_df.to_csv('./'+month_name+'/'+'uc_berkeleyprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'uc_berkeleyprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]
        
    df.to_csv('./'+month_name+'/'+'uc_berkeleyprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))

    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'

    df=df[['Professor Name','Professor Profile','Faculty']]

    df.to_csv('./'+month_name+'/'+'files/uc_berkeleyprofessors_prof_category.csv',index=False)
    print('Done for UC_berkeley')

def UIUC(options,month_name,keywords_ignore):

    # Initialize the WebDriver using ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://mechse.illinois.edu/people/faculty/all-faculty")
    time.sleep(5)

    # Parse the HTML content of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "name" (containing professor names)
    names = soup.find_all('div', class_='name')
    print(len(names))

    # Find all div elements with class "title" (containing professor titles/profiles)
    titles = soup.find_all('div', class_='title')
    print(len(titles))

    professor_names = []
    professor_profiles = []

    # Iterate over the professors and print their names and profiles
    for name, title in zip(names, titles):
        professor_names.append(name.text.strip())
        professor_profiles.append(title.text.strip())
        
    # Create a dataframe to store the data
    professors_df = pd.DataFrame({'Professor Name': professor_names, 'Professor Profile': professor_profiles})


    # Close the browser window
    driver.quit()

    professors_df.to_csv('./'+month_name+'/'+'UIUCprofessors.csv',index=False)

    df=pd.read_csv('./'+month_name+'/'+'UIUCprofessors.csv')

    for keyword in keywords_ignore:
        mask = df['Professor Profile'].str.contains(rf'\b{keyword}\b', regex=True,na=False)
        df=df[~mask]


    df.to_csv('./'+month_name+'/'+'UIUCprofessors_filter.csv',index=False)

    df['Professor Profile'] = df['Professor Profile'].fillna('')
    keys = ['Assistant Professor','ASST PROF','Asst Professor']
    keys1 = ['Associate Professor','ASSOC PROF','Assoc Professor']
    df['Assistant_Professor']=df['Professor Profile'].apply(lambda x: any(key in x for key in keys))
    df['Associate_Professor']=df['Professor Profile'].apply(lambda x: any(keyw in x for keyw in keys1))
    

    for i in df.index:
        if df.loc[i,'Assistant_Professor']==1:
            df.loc[i,'Faculty']='Assistant Professor'
        elif df.loc[i,'Associate_Professor']==1:
            df.loc[i,'Faculty']='Associate Professor'
        else:
            df.loc[i,'Faculty']='Professor'

    df=df[['Professor Name','Professor Profile','Faculty']]

    df.to_csv('./'+month_name+'/'+'files/UIUCprofessors_prof_category.csv',index=False)
    print('Done for UIUC')

    print('Done saving all files')

In [ ]:
Caltech(options,month_name,keywords_ignore)

In [ ]:
Georgia(options,month_name,keywords_ignore)

In [ ]:
Michigan(options,month_name,keywords_ignore)

In [ ]:
MIT(options,month_name,keywords_ignore)

In [ ]:
Purdue(options,month_name,keywords_ignore)

In [ ]:

Standford(options,month_name,keywords_ignore)

In [9]:
UC_Berkeley(options,month_name,keywords_ignore)

73
0 https://me.berkeley.edu/people/m-reza-alam/
1 https://me.berkeley.edu/people/francesco-borrelli/
2 https://me.berkeley.edu/people/van-p-carey/
3 https://me.berkeley.edu/people/james-casey/
4 https://me.berkeley.edu/people/chris-dames/
5 https://me.berkeley.edu/people/michael-frenklach/
6 https://me.berkeley.edu/people/michael-gollner/
7 https://me.berkeley.edu/people/kosa-goucher-lambert/
8 https://me.berkeley.edu/people/costas-grigoropoulos/
9 https://me.berkeley.edu/people/grace-x-gu/
10 https://me.berkeley.edu/people/roberto-horowitz/
11 https://me.berkeley.edu/people/alexis-kaminski/
12 https://me.berkeley.edu/people/ken-kamrin/
skipped i = 12
13 https://me.berkeley.edu/people/homayoon-kazerooni/
14 https://me.berkeley.edu/people/tony-m-keaveny/
15 https://me.berkeley.edu/people/kyriakos-komvopoulos/
16 https://me.berkeley.edu/people/liwei-lin/
17 https://me.berkeley.edu/people/fai-ma/
18 https://me.berkeley.edu/people/simo-makiharju/
19 https://me.berkeley.edu/people/philip-s

In [ ]:
options = webdriver.ChromeOptions()
#options.add_argument("--start-maximized")
#options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.page_load_strategy = "none"

UIUC(options,month_name,keywords_ignore)

#### - Sometimes This gives error in fetching data, so rerun the cell

#### --------------------------------------------------------------------------------------------------------------------------------------

#### It's Done!
#### You can check in your directory - "Month" folder and inside it "files" folder as filtered files for each website which are further used for creating graphs  